In [ ]:
# flake8: noqa
import warnings
import os

# Suppress noisy requests warnings.
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

# Extracting Information from Documents using Ray Datasets

In this example, we will show you how to run optical character recognition (OCR) on a set of documents and analyze the resulting text with the natural language processing library spaCy. To make it more interesting, we will run the analysis on the [LightShot](https://www.kaggle.com/datasets/datasnaek/lightshot) dataset. It is a large publicly available OCR dataset with a wide variety of different documents, all of them screen shots of various forms. It is easy to replace that dataset with your own data and adapt the example to your own use cases!

## Overview

This tutorial will cover:
 - Creating a Ray Dataset that represents the images in the dataset
 - Running the computationally expensive OCR process on each image in the dataset in parallel
 - Filtering out images that actually contain text
 - Performing various NLP operations on the text

## Walkthrough

Let's start by preparing the dependencies and downloading the dataset. You can download the dataset at [LightShot](https://www.kaggle.com/datasets/datasnaek/lightshot). Install the OCR software `tesseract` and extract the `archive.zip` file with the following commands:

In [ ]:
!sudo apt-get install tesseract-ocr
%pip install pytesseract
!sudo apt-get install -y unzip unrar
!unzip archive.zip
!unrar x LightShot13k.rar ~/LightShot13k/

Let's now import Ray and initialize a local Ray cluster. If you want to run OCR at a very large scale, you should run this workload on a multi-node cluster.

In [ ]:
# Import ray and initialize a local Ray cluster.
import ray
ray.init()

### Running the OCR software on the data

We first create a list `files` of absolute paths of the file names and then convert it into a Ray Dataset with the `ray.data.from_items` function. We can now run the `.map` function on this dataset of file names to run the actual OCR process on each file and convert the screen shots into text.

In [ ]:
import os
from glob import glob
files = glob(os.path.expanduser("~/LightShot13k/LightShot13k/*"))
ds = ray.data.from_items(files)
results = ds.map(lambda path: pytesseract.image_to_string(path))

Let us have a look at some of the data points with the `take` function.

In [ ]:
results.take()

### Process the extracted text data with spaCy

We can now process the extracted text with spaCy. Let's first make sure the libraries are installed:

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install spacy_langdetect

This is some code to determine the language of a piece of text:

In [ ]:
nlp = spacy.load('en_core_web_sm')

@Language.factory("language_detector")
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp.add_pipe('language_detector', last=True)
nlp("This is an English sentence")._.language

In order to run the code on the dataset, we have to use an actor since the `nlp` object is not serializable and we want to avoid having to recreate it for each individual sentence:

In [ ]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

class SpacyBatchInference:
    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')
        @Language.factory("language_detector")

        def get_lang_detector(nlp, name):
           return LanguageDetector()

        self.nlp.add_pipe('language_detector', last=True)

    def __call__(self, row):
        doc = self.nlp(row["value"])
        return doc._.language

results.limit(10).map(SpacyBatchInference, compute="actors")    

We can now get language statistics over the whole dataset:

In [ ]:
results.map(SpacyBatchInference, compute="actors").groupby("language").count().show()